In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap

from sklearn.datasets import make_classification, make_circles, load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix,
    roc_curve,
    roc_auc_score
)

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

---
## 3. PARTE I: SVM Lineal (Datos Linealmente Separables)

### 3.1 Generar datos sintéticos linealmente separables

Crearemos un dataset simple con 2 clases que se pueden separar con una línea recta.

In [ ]:
# Generar datos linealmente separables
np.random.seed(42)
X_linear, y_linear = make_classification(
    n_samples=100,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_clusters_per_class=1,
    class_sep=2.0,  # Separación clara entre clases
    random_state=42
)

# Visualizar los datos
plt.figure(figsize=(8, 6))
plt.scatter(X_linear[y_linear == 0][:, 0], X_linear[y_linear == 0][:, 1], 
            c='blue', label='Clase 0', alpha=0.7, edgecolors='k')
plt.scatter(X_linear[y_linear == 1][:, 0], X_linear[y_linear == 1][:, 1], 
            c='red', label='Clase 1', alpha=0.7, edgecolors='k')
plt.xlabel('Característica 1')
plt.ylabel('Característica 2')
plt.title('Dataset Linealmente Separable')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 3.2 Entrenar SVM Lineal

Usamos un **kernel lineal** (`kernel='linear'`) porque los datos son linealmente separables.

In [ ]:
# Dividir en train/test
X_train_lin, X_test_lin, y_train_lin, y_test_lin = train_test_split(
    X_linear, y_linear, test_size=0.3, random_state=42
)

# Escalar los datos (importante para SVM)
scaler_lin = StandardScaler()
X_train_lin_scaled = scaler_lin.fit_transform(X_train_lin)
X_test_lin_scaled = scaler_lin.transform(X_test_lin)

# Crear y entrenar SVM lineal
svm_linear = SVC(kernel='linear', C=1.0, random_state=42)
svm_linear.fit(X_train_lin_scaled, y_train_lin)

# Predicciones
y_pred_lin = svm_linear.predict(X_test_lin_scaled)

# Evaluación
print("=" * 50)
print("RESULTADOS SVM LINEAL")
print("=" * 50)
print(f"Accuracy: {accuracy_score(y_test_lin, y_pred_lin):.3f}")
print("\nReporte de Clasificación:")
print(classification_report(y_test_lin, y_pred_lin))

### 3.3 Visualizar el hiperplano y los vectores de soporte

Esta función dibuja:
- El hiperplano de separación (línea negra)
- Los márgenes (líneas punteadas)
- Los vectores de soporte (puntos rodeados)

In [ ]:
def plot_svm_decision_boundary(X, y, model, title="SVM Decision Boundary"):
    """
    Visualiza la frontera de decisión de un modelo SVM.
    Dibuja el hiperplano, los márgenes y resalta los vectores de soporte.
    """
    # Crear una malla de puntos para graficar la frontera
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))
    
    # Predecir en cada punto de la malla
    Z = model.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Graficar
    plt.figure(figsize=(10, 7))
    
    # Frontera de decisión y márgenes
    plt.contourf(xx, yy, Z, levels=[-1, 0, 1], alpha=0.2, 
                 colors=['blue', 'white', 'red'])
    plt.contour(xx, yy, Z, levels=[-1, 0, 1], linewidths=[1, 2, 1],
                colors=['blue', 'black', 'red'], linestyles=['--', '-', '--'])
    
    # Puntos de datos
    plt.scatter(X[y == 0][:, 0], X[y == 0][:, 1], 
                c='blue', label='Clase 0', alpha=0.6, edgecolors='k', s=80)
    plt.scatter(X[y == 1][:, 0], X[y == 1][:, 1], 
                c='red', label='Clase 1', alpha=0.6, edgecolors='k', s=80)
    
    # Vectores de soporte (resaltados)
    plt.scatter(model.support_vectors_[:, 0], model.support_vectors_[:, 1],
                s=200, linewidth=2, facecolors='none', edgecolors='green',
                label=f'Vectores de Soporte ({len(model.support_vectors_)})')
    
    plt.xlabel('Característica 1')
    plt.ylabel('Característica 2')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# Visualizar el modelo lineal
plot_svm_decision_boundary(
    X_train_lin_scaled, 
    y_train_lin, 
    svm_linear,
    title="SVM Lineal - Hiperplano y Vectores de Soporte"
)

print(f"\nNúmero de vectores de soporte: {len(svm_linear.support_vectors_)}")
print(f"De {len(X_train_lin_scaled)} puntos de entrenamiento, solo {len(svm_linear.support_vectors_)} determinan el hiperplano")

### 3.4 Efecto del parámetro C (regularización)

El parámetro **C** controla el trade-off entre:
- **C grande** → Margen estrecho, menos errores en entrenamiento (riesgo de overfitting)
- **C pequeño** → Margen amplio, permite algunos errores (más generalización)

Vamos a comparar diferentes valores de C:

In [ ]:
# Entrenar con diferentes valores de C
C_values = [0.01, 1.0, 100.0]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, C in enumerate(C_values):
    # Entrenar modelo
    svm_c = SVC(kernel='linear', C=C, random_state=42)
    svm_c.fit(X_train_lin_scaled, y_train_lin)
    
    # Crear malla
    x_min, x_max = X_train_lin_scaled[:, 0].min() - 1, X_train_lin_scaled[:, 0].max() + 1
    y_min, y_max = X_train_lin_scaled[:, 1].min() - 1, X_train_lin_scaled[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))
    
    Z = svm_c.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Graficar
    ax = axes[idx]
    ax.contourf(xx, yy, Z, levels=[-1, 0, 1], alpha=0.2, colors=['blue', 'white', 'red'])
    ax.contour(xx, yy, Z, levels=[-1, 0, 1], linewidths=[1, 2, 1],
               colors=['blue', 'black', 'red'], linestyles=['--', '-', '--'])
    
    ax.scatter(X_train_lin_scaled[y_train_lin == 0][:, 0], 
               X_train_lin_scaled[y_train_lin == 0][:, 1],
               c='blue', alpha=0.6, edgecolors='k', s=60)
    ax.scatter(X_train_lin_scaled[y_train_lin == 1][:, 0], 
               X_train_lin_scaled[y_train_lin == 1][:, 1],
               c='red', alpha=0.6, edgecolors='k', s=60)
    
    ax.scatter(svm_c.support_vectors_[:, 0], svm_c.support_vectors_[:, 1],
               s=150, linewidth=2, facecolors='none', edgecolors='green')
    
    ax.set_title(f'C = {C}\n{len(svm_c.support_vectors_)} vectores de soporte')
    ax.set_xlabel('Característica 1')
    ax.set_ylabel('Característica 2')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Observa cómo cambia el número de vectores de soporte y el margen con diferentes valores de C")

---
## 4. PARTE II: SVM con Kernel (Datos NO Linealmente Separables)

### 4.1 El problema: datos con patrón circular

Muchos problemas del mundo real no son linealmente separables. Por ejemplo, datos con patrón circular:

In [ ]:
# Generar datos con patrón circular (NO linealmente separables)
np.random.seed(42)
X_circles, y_circles = make_circles(
    n_samples=300,
    noise=0.1,
    factor=0.5,  # Radio relativo entre círculos
    random_state=42
)

# Visualizar
plt.figure(figsize=(8, 6))
plt.scatter(X_circles[y_circles == 0][:, 0], X_circles[y_circles == 0][:, 1],
            c='blue', label='Clase 0 (círculo exterior)', alpha=0.7, edgecolors='k')
plt.scatter(X_circles[y_circles == 1][:, 0], X_circles[y_circles == 1][:, 1],
            c='red', label='Clase 1 (círculo interior)', alpha=0.7, edgecolors='k')
plt.xlabel('Característica 1')
plt.ylabel('Característica 2')
plt.title('Dataset con Patrón Circular (NO linealmente separable)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.show()

print("¿Se puede separar con una línea recta? ❌ NO")
print("Necesitamos el Kernel Trick 🎩✨")

### 4.2 SVM Lineal vs SVM con Kernel RBF

Comparemos qué pasa si intentamos usar un kernel lineal vs un kernel RBF (Radial Basis Function).

**Kernel RBF**: Transforma los datos a un espacio de mayor dimensión donde SÍ son linealmente separables.

In [ ]:
# Dividir datos
X_train_circ, X_test_circ, y_train_circ, y_test_circ = train_test_split(
    X_circles, y_circles, test_size=0.3, random_state=42
)

# Escalar datos
scaler_circ = StandardScaler()
X_train_circ_scaled = scaler_circ.fit_transform(X_train_circ)
X_test_circ_scaled = scaler_circ.transform(X_test_circ)

# Entrenar con kernel LINEAL (va a fallar)
svm_linear_fail = SVC(kernel='linear', C=1.0, random_state=42)
svm_linear_fail.fit(X_train_circ_scaled, y_train_circ)
y_pred_lin_fail = svm_linear_fail.predict(X_test_circ_scaled)

# Entrenar con kernel RBF (va a funcionar bien)
svm_rbf = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm_rbf.fit(X_train_circ_scaled, y_train_circ)
y_pred_rbf = svm_rbf.predict(X_test_circ_scaled)

# Comparar resultados
print("=" * 60)
print("COMPARACIÓN: SVM LINEAL vs SVM RBF (datos circulares)")
print("=" * 60)
print("\n📉 SVM LINEAL (no apto para este problema):")
print(f"   Accuracy: {accuracy_score(y_test_circ, y_pred_lin_fail):.3f}")

print("\n📈 SVM con Kernel RBF (diseñado para este problema):")
print(f"   Accuracy: {accuracy_score(y_test_circ, y_pred_rbf):.3f}")
print("\nReporte de Clasificación (RBF):")
print(classification_report(y_test_circ, y_pred_rbf))

### 4.3 Visualizar las fronteras de decisión

Veamos gráficamente por qué el kernel RBF funciona mucho mejor:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Función para graficar frontera de decisión
def plot_decision_boundary_circles(ax, X, y, model, title):
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))
    
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    ax.contourf(xx, yy, Z, alpha=0.3, cmap='RdBu')
    ax.scatter(X[y == 0][:, 0], X[y == 0][:, 1], c='blue', alpha=0.6, edgecolors='k', s=60)
    ax.scatter(X[y == 1][:, 0], X[y == 1][:, 1], c='red', alpha=0.6, edgecolors='k', s=60)
    ax.scatter(model.support_vectors_[:, 0], model.support_vectors_[:, 1],
               s=150, linewidth=2, facecolors='none', edgecolors='green',
               label=f'Vectores de soporte ({len(model.support_vectors_)})')
    ax.set_xlabel('Característica 1')
    ax.set_ylabel('Característica 2')
    ax.set_title(title)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.axis('equal')

# Graficar ambos modelos
plot_decision_boundary_circles(
    axes[0], X_train_circ_scaled, y_train_circ, svm_linear_fail,
    f"SVM Lineal\nAccuracy: {accuracy_score(y_test_circ, y_pred_lin_fail):.3f}"
)

plot_decision_boundary_circles(
    axes[1], X_train_circ_scaled, y_train_circ, svm_rbf,
    f"SVM con Kernel RBF\nAccuracy: {accuracy_score(y_test_circ, y_pred_rbf):.3f}"
)

plt.tight_layout()
plt.show()

print("\n💡 Observa cómo el kernel RBF puede crear una frontera circular")
print("   mientras que el lineal solo puede trazar una línea recta")

### 4.4 El parámetro gamma en Kernel RBF

**Gamma** controla cuánta influencia tiene cada punto de entrenamiento:
- **Gamma alto** → Influencia local (riesgo de overfitting)
- **Gamma bajo** → Influencia amplia (frontera más suave)

Vamos a comparar:

In [ ]:
gamma_values = [0.1, 1.0, 10.0]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, gamma in enumerate(gamma_values):
    # Entrenar modelo
    svm_g = SVC(kernel='rbf', C=1.0, gamma=gamma, random_state=42)
    svm_g.fit(X_train_circ_scaled, y_train_circ)
    y_pred_g = svm_g.predict(X_test_circ_scaled)
    
    # Graficar
    plot_decision_boundary_circles(
        axes[idx], X_train_circ_scaled, y_train_circ, svm_g,
        f"Gamma = {gamma}\nAccuracy Test: {accuracy_score(y_test_circ, y_pred_g):.3f}"
    )

plt.tight_layout()
plt.show()

print("\n📊 Conclusión sobre Gamma:")
print("   - Gamma muy bajo (0.1): Frontera demasiado suave, underfitting")
print("   - Gamma medio (1.0): Balance óptimo")
print("   - Gamma muy alto (10.0): Frontera muy irregular, overfitting")

---
## 5. PARTE III: Caso Real - Detección de Cáncer de Mama

Ahora aplicaremos SVM a un dataset real: **Breast Cancer Wisconsin**.

**Objetivo**: Clasificar tumores como benignos (0) o malignos (1) basándose en características de las células.

In [ ]:
# Cargar dataset real
cancer = load_breast_cancer()
X_cancer = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y_cancer = pd.Series(cancer.target, name="diagnosis")

print("=" * 60)
print("DATASET: BREAST CANCER WISCONSIN")
print("=" * 60)
print(f"Número de muestras: {len(X_cancer)}")
print(f"Número de características: {X_cancer.shape[1]}")
print(f"\nDistribución de clases:")
print(f"  Clase 0 (Maligno): {(y_cancer == 0).sum()} ({(y_cancer == 0).sum()/len(y_cancer)*100:.1f}%)")
print(f"  Clase 1 (Benigno): {(y_cancer == 1).sum()} ({(y_cancer == 1).sum()/len(y_cancer)*100:.1f}%)")
print(f"\nPrimeras características:")
print(X_cancer.head())

### 5.1 Preprocesamiento y división de datos

In [ ]:
# Dividir en train/test
X_train_cancer, X_test_cancer, y_train_cancer, y_test_cancer = train_test_split(
    X_cancer, y_cancer, test_size=0.2, random_state=42, stratify=y_cancer
)

# Escalar características (CRÍTICO para SVM)
scaler_cancer = StandardScaler()
X_train_cancer_scaled = scaler_cancer.fit_transform(X_train_cancer)
X_test_cancer_scaled = scaler_cancer.transform(X_test_cancer)

print("Datos preparados:")
print(f"  Train: {X_train_cancer_scaled.shape}")
print(f"  Test: {X_test_cancer_scaled.shape}")

### 5.2 Búsqueda de hiperparámetros óptimos con GridSearchCV

Usaremos validación cruzada para encontrar los mejores valores de C y gamma:

In [ ]:
# Definir grid de hiperparámetros
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['rbf']
}

# GridSearchCV con validación cruzada
grid_search = GridSearchCV(
    SVC(random_state=42),
    param_grid,
    cv=5,  # 5-fold cross-validation
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

print("Buscando mejores hiperparámetros...")
grid_search.fit(X_train_cancer_scaled, y_train_cancer)

print("\n" + "=" * 60)
print("RESULTADOS DE GRID SEARCH")
print("=" * 60)
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor score CV: {grid_search.best_score_:.3f}")

### 5.3 Evaluar el modelo optimizado

In [ ]:
# Mejor modelo
best_svm = grid_search.best_estimator_

# Predicciones
y_pred_cancer = best_svm.predict(X_test_cancer_scaled)
y_pred_proba_cancer = best_svm.decision_function(X_test_cancer_scaled)

# Métricas
print("\n" + "=" * 60)
print("EVALUACIÓN EN TEST SET")
print("=" * 60)
print(f"Accuracy: {accuracy_score(y_test_cancer, y_pred_cancer):.3f}")
print("\nReporte de Clasificación:")
print(classification_report(y_test_cancer, y_pred_cancer, 
                          target_names=['Maligno', 'Benigno']))

# Matriz de confusión
cm_cancer = confusion_matrix(y_test_cancer, y_pred_cancer)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_cancer, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Maligno', 'Benigno'],
            yticklabels=['Maligno', 'Benigno'])
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión - SVM en Breast Cancer')
plt.show()

print(f"\n📊 Interpretación:")
print(f"   - Verdaderos Negativos (TN): {cm_cancer[0,0]} (malignos correctamente identificados)")
print(f"   - Falsos Positivos (FP): {cm_cancer[0,1]} (benignos clasificados como malignos)")
print(f"   - Falsos Negativos (FN): {cm_cancer[1,0]} (malignos clasificados como benignos ⚠️ CRÍTICO)")
print(f"   - Verdaderos Positivos (TP): {cm_cancer[1,1]} (benignos correctamente identificados)")

### 5.4 Curva ROC y AUC

In [ ]:
# Calcular ROC
fpr, tpr, thresholds = roc_curve(y_test_cancer, y_pred_proba_cancer)
roc_auc = roc_auc_score(y_test_cancer, y_pred_proba_cancer)

# Graficar
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, 
         label=f'SVM RBF (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', 
         label='Clasificador Aleatorio (AUC = 0.5)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curva ROC - SVM para Detección de Cáncer')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n🎯 AUC = {roc_auc:.3f}")
print("   AUC > 0.95 indica excelente capacidad de discriminación")

### 5.5 Importancia de características (aproximada)

SVM no proporciona importancia de características directamente, pero podemos aproximarla con los coeficientes del modelo lineal:

In [ ]:
# Entrenar un SVM lineal para ver coeficientes
svm_linear_cancer = SVC(kernel='linear', C=1.0, random_state=42)
svm_linear_cancer.fit(X_train_cancer_scaled, y_train_cancer)

# Obtener coeficientes
coef = pd.Series(
    np.abs(svm_linear_cancer.coef_[0]), 
    index=X_cancer.columns
).sort_values(ascending=False)

# Visualizar top 10
plt.figure(figsize=(10, 6))
coef[:10].plot(kind='barh', color='steelblue')
plt.xlabel('Importancia Absoluta (|coeficiente|)')
plt.title('Top 10 Características Más Importantes (SVM Lineal)')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\n📋 Top 5 características más influyentes:")
for i, (feature, importance) in enumerate(coef[:5].items(), 1):
    print(f"   {i}. {feature}: {importance:.3f}")

---
## 6. Comparación de Kernels

Comparemos el rendimiento de diferentes kernels en el dataset de cáncer:

In [ ]:
kernels = ['linear', 'rbf', 'poly', 'sigmoid']
results = []

for kernel in kernels:
    # Entrenar modelo
    svm_k = SVC(kernel=kernel, C=1.0, random_state=42, gamma='scale')
    svm_k.fit(X_train_cancer_scaled, y_train_cancer)
    
    # Predecir
    y_pred_k = svm_k.predict(X_test_cancer_scaled)
    
    # Guardar resultados
    accuracy = accuracy_score(y_test_cancer, y_pred_k)
    results.append({
        'Kernel': kernel,
        'Accuracy': accuracy,
        'N° Vectores de Soporte': len(svm_k.support_vectors_)
    })
    
# Mostrar resultados
results_df = pd.DataFrame(results).sort_values('Accuracy', ascending=False)
print("\n" + "=" * 60)
print("COMPARACIÓN DE KERNELS")
print("=" * 60)
print(results_df.to_string(index=False))

# Visualizar
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
ax1.bar(results_df['Kernel'], results_df['Accuracy'], color='steelblue', alpha=0.7)
ax1.set_ylabel('Accuracy')
ax1.set_title('Accuracy por Kernel')
ax1.set_ylim([0.9, 1.0])
ax1.grid(True, alpha=0.3, axis='y')
for i, v in enumerate(results_df['Accuracy']):
    ax1.text(i, v + 0.005, f'{v:.3f}', ha='center', va='bottom', fontweight='bold')

# Número de vectores de soporte
ax2.bar(results_df['Kernel'], results_df['N° Vectores de Soporte'], color='coral', alpha=0.7)
ax2.set_ylabel('Número de Vectores de Soporte')
ax2.set_title('Complejidad del Modelo (Vectores de Soporte)')
ax2.grid(True, alpha=0.3, axis='y')
for i, v in enumerate(results_df['N° Vectores de Soporte']):
    ax2.text(i, v + 5, str(v), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

---
## 7. Conclusiones

### ✅ Lo que hemos aprendido:

1. **SVM Lineal:**
   - Funciona perfectamente cuando los datos son linealmente separables
   - Busca el hiperplano con el **margen máximo**
   - Solo depende de los **vectores de soporte**

2. **Kernel Trick:**
   - Solución elegante para datos NO linealmente separables
   - **Kernel RBF**: Transforma implícitamente a un espacio de dimensión infinita
   - **Gamma**: Controla la influencia local/global de cada punto

3. **Parámetro C:**
   - Trade-off entre margen amplio y clasificación correcta
   - C alto → Menos errores en training (riesgo overfitting)
   - C bajo → Margen más amplio (mejor generalización)

4. **Aplicación Real:**
   - SVM es excelente para datasets médicos (alta precisión)
   - GridSearchCV es esencial para encontrar hiperparámetros óptimos
   - **Siempre escalar los datos** antes de usar SVM

### 🎯 Ventajas de SVM:
- ✅ Muy efectivo en espacios de alta dimensión
- ✅ Eficiente en memoria (solo usa vectores de soporte)
- ✅ Versátil gracias a los diferentes kernels
- ✅ Funciona bien con datasets pequeños/medianos

### ⚠️ Desventajas de SVM:
- ❌ Lento en datasets muy grandes (n > 10,000)
- ❌ Requiere escalado de características
- ❌ Difícil interpretabilidad (especialmente con kernels no lineales)
- ❌ Sensible a la selección de hiperparámetros

### 📚 Próximos Pasos:
- Experimentar con otros kernels (polynomial, sigmoid)
- Probar SVM en problemas multiclase
- Combinar SVM con técnicas de reducción de dimensionalidad (PCA)
- Comparar SVM con otros algoritmos (Random Forest, XGBoost)